# Probability model perspective of VAE

## 1. Bayesian probability <br>

확률모델에 입각한 VAE의 설명은 베이지안을 반드시 짚고 넘어가야 한다. <br>

먼저 식을 바라보자. <br>

$p(z|x) = \dfrac{p(x|z)p(z)}{p(x)}$ <br>

- $p(z|x)$ : Posterior probability<br>

- $p(x|z)$ : likelihood <br>

- $p(z)$ : prior probability <br>

- $p(x)$ : evidence. (marginal probability) <br>

VAE에서 $z = $ latent variable 이고 $x = data$ 이다. <br> 

## 2. 정보의 양.

특정 분포를 따르는 확률변수 X를 관측하는데, random sample에서 흔히 나올 수 있는 것들 (p(x)가 높은 쪽에서 나오는 것들) 은 정보의 양이 적다. <br>

그것이 정보이다. <br>

따라서 이를 표현하기 위해 정보를 나타내는 방식을 고안해 내었는데, 다음과 같다 : <br>

$h(x) = -log p(x)$

여기서 $h$, 즉 정보의 단위는 log의 밑에 따라 규정이 되는데,  2 = bit, e = nat이다. <br>

주로 KL-divergence에서는 nat이 사용된다. <br>

## 3. 엔트로피

엔트로피는 해당 변수가 가지는 (혹은 전송될 때 필요한) 평균 정보량(혹은 비트 내지는 nat의 수) 이다. <br>

수식으로 표현하면 $H(x) = \Sigma p(x)h(x) $ 이다. <br> 

예를 들면, 8개의 다른 값을 가질 수 있는 X 가 Unif dist'n을 따른다면 <br>
$H_2(X) = 3$ 이다. <br>
즉, X가 전송될 때 필요한 bit 수가 3개이며, X라는 변수가 가지는 평균 정보량이 3이다. <br>

## 4. KL divergence.

궁금한 확률분포 $X ~ p(x)$ 가 있다. <br>

이를 approximate dist'n q(x)로 예측하고 싶은데, 이 두 분포의 실질적인 가까움을 '정보량'으로 알아보고 싶은 것이다. <br>

$KL(q(x)||p(x)) = - \int_{}q(x)lnp(x)dx - (-\int_{}q(x)lnq(x)dx)$ <br>

이를 말로 풀면, q(x)에 맞춰진 정보량을 보내야 할 자리에 p(x)를 넣으면 정보량이 어떻게 변하는지를 넣으면 정보량이 어떻게 변하는가를 나타낸다. <br>

## 5. VAE

$  P(x,z) = p(z) * p(x|z) $ <br>
$ Z$ ~ $N(0,1) $ <br>

우리가 여기서 가장 궁금해하는 것은 $p(z|x)$, 즉 posterior probability 이다. <br>

data가 given일 때, 가장 좋은 latent variable을 유추하는 것이다. <br>

이를 위해서는 위의 bayesian model을 사용하는데, 그렇다면 각 likelihood, prior prob, evidence는 어떻게 구할까. <br>

evidence는 보통 $p(x) = \int_{}{}p(x|z)p(z)dz $ 로 계산하는데, z에 따라 너무나 많은 계산량이 들어가므로 $p(z|x)$ 자체를 approximate 한다. <br>

그 방식은 바로 $q_{\lambda}(z|x)$ 와의 KL-divergence로 구한다. <br>
(여기서 $\lambda$는 각 datapoint $x_i$에서의 $Z$의 분포의 모수를 의미한다) <br>

이를 수식으로 나타내면 : <br>
$arg min_{\lambda} KL(q_{\lambda}(z|x)||p(z|x))$  <br>

= $arg min_{\lambda} E_q(logq_{\lambda}(z|x)) - E_q(logq_{\lambda}(logp(x,z)) + logp(x)$

여기서 evidence p(x)는 구하는데 문제가 있다. <br>

이를 해결하기 위하여 ELBO라는 것이 사용된다. <br>

$ ELBO(\lambda) =  E_q(logq_{\lambda}(logp(x,z)) - E_q(logq_{\lambda}(z|x)) $ <br>

여기서 KL을 min 하는 것이 아닌 ELBO를 max하는 방향으로 선회한다.

그래서 loss function 자체가 ELBO가 되는 것이다. 이를 다시 표기하면 : <br>

$ELBO_i(\theta,\phi) = E_{q_{\theta}}[logp_{\phi}(x_i|z)] - KL(q_{\theta}(z|x_i)||p(z))$

각 data point $x_i$마다 z가 다르므로(독립적), ELBO를 각 datapoint i 마다 분해해서 minibatch 등을 할 수 있다. <br>

위의 식에서 posterior part $q_\theta (z|x,\lambda)$ 는 encoding part 이고, <br>
likelihood $p_\phi(x|z)$ 는 decoding part 이므로 각각을 gradient ascent(maximizing)한다. <br>

결국 해석은 이렇다. <br>

수식적으로 KL part는 $q_\lambda(z|x)$를 $p(z|x)$에 계산상의 용이를 위하여approximate 시킨다고 이해하면 되고, <br>
감정적으로는 $p_\theta$ part 가 reconstruction loss, KL이 regularizer 라고 이해하면 편하다. <br>

위의 loss를 두 가지로 나눠서 보자. 첫 번째는 encoding part이다. <br>
encoding part의 모수, 여기서는 $\lambda$ or $\theta$로 작성되었다) 는 variational inference(w\ KL divergence) 기법을 통해 구해진다. <br>

두 번째는 decoding part. <br>
이는 정보량의 평균을 통해 구해진다. 그래서 이름은 EM(expectation maximization이라고 부른다.)
 
마지막으로, $q_\lambda(z|x)$에서 gradient를 구하기 위해서는 z 가 random variable이 아니라 x의 모수에 종속적인 형태여야 미분이 가능하다. <br>

따라서 위에서 z ~ N(0,1)이라고 했던 것을, 미분을 위해 $z = \mu + \sigma * \epsilon$ where $\epsilon$~ N(0,1) 이다.